## Optimizing Newspaper and Magazine Shelf Space and Pricing for Maximum Profit

### Decision Variables

* $x_A, x_B, x_m \in \Z_+$ : indicating the number of Newspaper $A$, Newspaper $B$, and Magazines purchased.
* $y_A, y_B, y_m, y_p \in \Z_+$ : indicating the number of Newspaper $A$ sold individually, Newspaper $B$ sold, Magazines sold individually, and “bundle” (newspaper $A$ + magazine) units sold.
* $z_A, z_B \in \Z_+ $ : indicating the number of unsold/returned copies of Newspaper $A$ and $B$.

### Parameters

* $c_A, c_B, c_m$ : Purchase (cost) prices for $A,B$ and magazine, respectively. (B is cheaper)
* $p_A​,p_B​,p_m​$ : Selling prices (per unit) for $A, B$ and magazine, respectively.
* $r_A, r_B$ : Refunds (return values) for unsold copies of $A$ and $B$.
    - $r_A =  0.40 c_A$
    - $r_B = 0.10 c_B$ 
* $d_A, d_B, d_m, d_p$ :  Demands (maximum potential sales) for $A$, $B$, magazine, and the bundle, respectively.
* $C$: Total Budget Available
* $S_n, S_m$: Shelf capacities for newspapers and magazines, respectively.
* Discount: 8% off $(p_A+p_m)$ for each bundle sale, so bundle revenue is $0.92 (pA+pm)$.

### Constraints

1. Budget Constraints

    $c_A x_A + c_B x_B + c_m x_m \leq C$

2. Shelf-Space Constraints

    * Newspapers (both $A$ and $B$ share the same shelf):

        $x_A +  x_B  \leq S_n$
    
    * Magazines have their own shelf:
    
        $ x_m \leq S_m$

3. Demand Constraints:

    $y_A \leq d_A$, <br>
    $y_B \leq d_B$, <br>
    $y_m \leq d_m$, <br>
    $y_p \leq d_p$, <br>

4. Purchase‐to‐Sale (and Returns) Balances

    * Newspaper $B$

        $y_B +  z_B  = x_B$
    
    * Newspaper $A$:
    
        $y_A + y_p + z_A  = x_A$ (the bundle $y_p$ also uses up an A)

    * Magazine (assuming no returns, just leftover is wasted or not allowed):
    
        $y_m + y_p  \leq x_m$

5. Nonnegativity

    $x_A​,x_B​,x_m​,y_A​,y_B​,y_m​,y_p​,z_A​,z_B​ \geq 0$


    



### Objective Function

Maximize total profit, computed as:

* Revenue from individual newspaper and magazine sales:

    $p_A​ y_A ​+ p_B​ y_B ​+ p_m​ y_m​$

* Revenue from bundle sales at 8% discount:

    $0.92 (p_A + p_m)​$

* Refund for unsold units of $A$ and $B$:

    $r_A ​z_A ​+ r_B ​z_B$

* Minus the total purchase cost:

    $−c_A ​x_A​ − c_B​ x_B​ − c_m​ x_m​​$


Putting it all together:

\begin{equation}

max \ \ p_A​ y_A ​+ p_B​ y_B ​+ p_m​ y_m + 0.92 (p_A + p_m) + r_A ​z_A ​+ r_B ​z_B −c_A ​x_A​ − c_B​ x_B​ − c_m​ x_m​

\end{equation}


In [1]:
import cplex 
from cplex.exceptions import CplexError
from docplex.mp.model import Model
import time
import pandas as pd

In [2]:
m = Model('newsvendor_shelf', log_output=True)

Defining Decision Variables

In [3]:
xA = m.continuous_var(name='xA', lb=0)
xB = m.continuous_var(name='xB', lb=0)
xm = m.continuous_var(name='xm', lb=0)
yA = m.continuous_var(name='yA', lb=0)
yB = m.continuous_var(name='yB', lb=0)
ym = m.continuous_var(name='ym', lb=0)
yp = m.continuous_var(name='yp', lb=0)
zA = m.continuous_var(name='zA', lb=0)
zB = m.continuous_var(name='zB', lb=0)


Example Parameter Values

In [4]:
# budget
C = 300

# costs
cA = 5
cB = 4.5
cm = 3

# selling prices
pA = 8
pB = 8
pm = 5

# demand
dA = 20
dB = 20
dm = 15
dp = 10

# shelf capacities
Sn = 30
Sm = 20

# return values
rA = 0.40 * cA
rB = 0.10 * cB

# pack discount
pd = 0.92 * (pA + pm)


Defining Constraints

In [5]:
# budget constraint
m.add_constraint(xA * cA + xB * cB + xm * cm <= C)

# shelf capacity constraints
m.add_constraint(xA + xB <= Sn)
m.add_constraint(xm <= Sm)

# demand constraints
m.add_constraint(yA <= dA)
m.add_constraint(yB <= dB)
m.add_constraint(ym <= dm)
m.add_constraint(yp <= dp)

# purchase to sale balances 
m.add_constraint(yA + yp + zA == xA)
m.add_constraint(yB + zB == xB)
m.add_constraint(ym + yp <= xm)


docplex.mp.LinearConstraint[](ym+yp,LE,xm)

In [6]:
obj_fn = pA * yA + pB * yB + pm * ym + 0.92 * (pA + pm) * yp + rA * zA + rB * zB - cA * xA - cB * xB - cm * xm
m.set_objective('max', obj_fn)

In [7]:
m.solve(log_output=True)

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 6 rows and 2 columns.
Aggregator did 2 substitutions.
Reduced LP has 2 rows, 5 columns, and 6 nonzeros.
Presolve time = 0.00 sec. (0.01 ticks)

Iteration log . . .
Iteration:     1   Dual infeasibility =             0.000000
Iteration:     2   Dual objective     =           139.600000


docplex.mp.solution.SolveSolution(obj=134.8,values={xA:10,xB:20,xm:20,yA..

In [8]:
m.solve_status

<JobSolveStatus.OPTIMAL_SOLUTION: 2>

In [9]:
#print model
print(m.export_to_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: newsvendor_shelf

Maximize
 obj: - 5 xA - 4.500000000000 xB - 3 xm + 8 yA + 8 yB + 5 ym
      + 11.960000000000 yp + 2 zA + 0.450000000000 zB
Subject To
 c1: 5 xA + 4.500000000000 xB + 3 xm <= 300
 c2: xA + xB <= 30
 c3: xm <= 20
 c4: yA <= 20
 c5: yB <= 20
 c6: ym <= 15
 c7: yp <= 10
 c8: yA + yp + zA - xA = 0
 c9: yB + zB - xB = 0
 c10: ym + yp - xm <= 0

Bounds
End



In [10]:
# objective value
print(m.objective_value)

134.8


In [11]:
# decision variables values
print(m.solution.get_value(xA))
print(m.solution.get_value(xB))
print(m.solution.get_value(xm))
print(m.solution.get_value(yA))
print(m.solution.get_value(yB))
print(m.solution.get_value(ym))
print(m.solution.get_value(yp))
print(m.solution.get_value(zA))
print(m.solution.get_value(zB))

10.0
20.0
20.0
5.0
20.0
15.0
5.0
0
0


### Considering Samples Demands...

In [12]:
import pandas as pd
import numpy as np

In [13]:
m_demand = Model('newsvendor_shelf', log_output=True)

In [14]:
# Load the data from the Excel file
data = pd.read_excel('./samples.xlsx')

In [15]:
data.sample(5)

,newspaper1,newspaper2,magazine,pack
356,14,18,102,15
781,15,26,48,25
37,5,32,20,35
718,19,29,36,25
879,35,43,59,6


In [16]:
xA = m_demand.continuous_var(name='xA', lb=0)
xB = m_demand.continuous_var(name='xB', lb=0)
xm = m_demand.continuous_var(name='xm', lb=0)
zA = m_demand.continuous_var(name='zA', lb=0)
zB = m_demand.continuous_var(name='zB', lb=0)

In [17]:
yA = m_demand.continuous_var_list(len(data.index), lb=-np.inf, name='yA')
yB = m_demand.continuous_var_list(len(data.index), lb=-np.inf, name='yB')
ym = m_demand.continuous_var_list(len(data.index), lb=-np.inf, name='ym')
yp = m_demand.continuous_var_list(len(data.index), lb=-np.inf, name='yp')


In [18]:
# getting demands
dA = data['newspaper1']
dB = data['newspaper2']
dm = data['magazine']
dp = data['pack']

In [19]:
# budget constraint
m_demand.add_constraint(xA * cA + xB * cB + xm * cm <= C)

# shelf capacity constraints
m_demand.add_constraint(xA + xB <= Sn)
m_demand.add_constraint(xm <= Sm)

# demand constraints
for i in range(len(data.index)):
    m_demand.add_constraint(yA[i] <= dA[i])
    m_demand.add_constraint(yB[i] <= dB[i])
    m_demand.add_constraint(ym[i] <= dm[i])
    m_demand.add_constraint(yp[i] <= dp[i])

    m_demand.add_constraint(yA[i] + yp[i] + zA == xA)
    m_demand.add_constraint(yB[i] + zB == xB)
    m_demand.add_constraint(ym[i] + yp[i] <= xm)


In [20]:
obj_fn_demand = sum([pA * yA[i] + pB * yB[i] + pm * ym[i] + 0.92 * (pA + pm) * yp[i] + rA * zA + rB * zB - cA * xA - cB * xB - cm * xm for i in range(len(data.index))])
m_demand.set_objective('max', obj_fn_demand)

In [21]:
m_demand.solve(log_output=True)

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Parallel mode: deterministic, using up to 16 threads for concurrent optimization:
 * Starting dual Simplex on 1 thread...
 * Starting Barrier on 14 threads...
 * Starting primal Simplex on 1 thread...
Tried aggregator 1 time.
LP Presolve eliminated 4001 rows and 0 columns.
Reduced LP has 3002 rows, 4005 columns, and 10005 nonzeros.
Presolve time = 0.01 sec. (4.79 ticks)
Symmetry aggregator did 1948 additional substitutions.

Iteration log . . .
Iteration:     1   Scaled dual infeas =          2258.000000
Iteration:   253   Dual objective     =        722899.320000
Iteration:   660   Dual objective     =        566440.960000
Iteration:   936   Dual objective     =        481697.120000
Iteration:  1195   Dual objective     =        369540.120000
Iteration:  1324   Dual objective     =        324716.520000
Iteration:  1497   Dual objective     =        278011.720000
Iteration:  1669   

docplex.mp.solution.SolveSolution(obj=105288,values={xA:11,xB:7,xm:20,yA..

In [22]:
m_demand.solve_status

<JobSolveStatus.OPTIMAL_SOLUTION: 2>

In [25]:
y_Av = [y.solution_value for y in yA]
y_Av


[13.0,
 21.0,
 22.0,
 18.0,
 17.0,
 13.0,
 10.0,
 7.0,
 7.0,
 8.0,
 8.0,
 17.0,
 14.0,
 21.0,
 20.0,
 16.0,
 23.0,
 24.0,
 13.0,
 10.0,
 12.0,
 12.0,
 11.0,
 26.0,
 18.0,
 34.0,
 24.0,
 36.0,
 32.0,
 19.0,
 21.0,
 28.0,
 22.0,
 40.0,
 28.0,
 13.0,
 18.0,
 5.0,
 14.0,
 15.0,
 14.0,
 15.0,
 6.0,
 23.0,
 9.0,
 17.0,
 19.0,
 26.0,
 26.0,
 10.0,
 22.0,
 9.0,
 32.0,
 6.0,
 21.0,
 17.0,
 1.0,
 27.0,
 18.0,
 34.0,
 29.0,
 9.0,
 9.0,
 16.0,
 10.0,
 37.0,
 13.0,
 17.0,
 43.0,
 21.0,
 18.0,
 21.0,
 19.0,
 35.0,
 10.0,
 31.0,
 18.0,
 11.0,
 25.0,
 11.0,
 18.0,
 22.0,
 16.0,
 15.0,
 16.0,
 7.0,
 18.0,
 22.0,
 40.0,
 20.0,
 30.0,
 4.0,
 45.0,
 18.0,
 25.0,
 19.0,
 38.0,
 15.0,
 16.0,
 21.0,
 18.0,
 13.0,
 17.0,
 30.0,
 10.0,
 19.0,
 7.0,
 15.0,
 41.0,
 11.0,
 22.0,
 16.0,
 17.0,
 25.0,
 12.0,
 26.0,
 10.0,
 15.0,
 11.0,
 25.0,
 18.0,
 22.0,
 14.0,
 40.0,
 12.0,
 23.0,
 22.0,
 34.0,
 19.0,
 18.0,
 13.0,
 9.0,
 25.0,
 30.0,
 21.0,
 13.0,
 29.0,
 39.0,
 17.0,
 16.0,
 13.0,
 11.0,
 9.0,
 17.0,
 17.0,
 3

In [26]:
xA.solution_value

11.0

In [27]:
print(m_demand.objective_value)

105287.88000000018
